# <center>**Import necessary libraries**</center>


In [702]:
import pathlib
import pandas as pd
import math
import design_functions.rebar_information as rebar_func

## <center>Import and read the desired beam flexural and shear reinforcement excel file from ETABS</center>

In [703]:
excel_file = pathlib.Path('excel_files\example_analysis.xlsx')
initial_df = pd.read_excel(excel_file, sheet_name=None)
pd.set_option('display.max_rows', None)

## <center>Because the excel sheet has two sheets, flexural and shear, they need to be read individually</center>

## <center>Delete unnecessary rows and columns for simplicity</center>

V1 of respective df's = first two rows are deleted<br>
V2 of respective df's = unique name, negative moment, negative combo, positive moment, and positive combo columns are removed.<br>
V3 of respective df's = each unique beam is consolidated to extract information such as section size, unqiue name.<br>
V4 of respective df's = ETABs section column is removed.<br>
V5 of respective df's = V2 df but all columns are removed except bottom area reinforcement and top area reinforcement<br>
V6 of respective df's = calculation DF including 100 indices and all rebar requirements<br>
For shear, identifying columns are removed as they are not needed. Additionally, unique beams DF doesn't need to be made as they can be taken from flexural DF. Therefore V2 may be primarily used

In [704]:
v1_flexural_df = initial_df['Sheet1'].drop([0,1])
v1_shear_df = initial_df['Sheet2'].drop([0,1])

In [705]:
v2_flexural_df = v1_flexural_df.drop(['Unnamed: 2', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
v2_shear_df = v1_shear_df.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 11', 'Unnamed: 12'], axis=1)

In [706]:
v3_flexural_df = v2_flexural_df.iloc[::3]

## <center>Create width and depth columns to utilise in calculations</center>

In [707]:
v2_flexural_df.reset_index(drop=True, inplace=True)

In [708]:
v3_flexural_df.insert(3, 'Width (mm)', None)
v3_flexural_df.insert(4, 'Depth (mm)', None)

In [709]:
#this function cleans the cell of unnamed: 3 column to provide the width of each respective beam.
def clean_width_dimensions(width):
    width_list = list(width) # turn string into list of individual indexes
    width_list = [el.lower() for el in width_list] #use list comprehension to turn list into lower case values
    excluded_values = ['p', 't', 'b', '-', '_', 'c', '/'] #create list of excluded indices
    v1_width_list = [ex for ex in width_list if ex not in excluded_values] #use list comprehension to return list excluding indices
    index_list = v1_width_list.index('x') #index the list to x to retrieve required width
    v2_width_list = v1_width_list[:index_list] #slice the width list to the index x
    true_width = ''.join(v2_width_list) #join the list into a string
    return int(true_width) #turn string into int so it can be used in other functions

In [710]:
v3_flexural_df.loc[:, 'Width (mm)'] = v3_flexural_df['Unnamed: 3'].apply(clean_width_dimensions)

In [711]:
#this function cleans the cell of unnamed: 3 column to provide the depth of each respective beam.
#this function follows the same steps as clean_width_dimensions function
def clean_depth_dimensions(depth):
    depth_list = list(depth)
    depth_list = [el.lower() for el in depth_list]
    excluded_values = ['p','t', 'b', '-', '_', 'c', '/']
    v1_depth_list = [ex for ex in depth_list if ex not in excluded_values]
    index_list = v1_depth_list.index('x')
    v2_depth_list = v1_depth_list[1+index_list:-4]
    true_depth = ''.join(v2_depth_list)
    return int(true_depth)

In [712]:
v3_flexural_df.loc[:, 'Depth (mm)'] = v3_flexural_df['Unnamed: 3'].apply(clean_depth_dimensions)

## <center>With the width and depth defined, remove the ETABS section, location, and rebar area columns</center>

In [713]:
v4_flexural_df = v3_flexural_df.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 7', 'Unnamed: 10'], axis=1)

In [714]:
v4_flexural_df = v4_flexural_df.rename(columns={'Unnamed: 1': 'ETABS beam ID'})
v2_shear_df = v2_shear_df.rename(columns={'Unnamed: 7': 'Shear rebar area (mm2/m)', 'Unnamed: 10': 'Shear tension rebar area (mm2/m)', 'Unnamed: 13': 'Longitudinal tension rebar area (mm2)'})

## <center>Create Dataframe to store beam reinforcement schedule</center>

In [715]:
# Create dataframe which reflects column headings
columns = pd.MultiIndex.from_tuples([
    ('Storey', ''),
    ('Etabs ID', ''),
    ('Dimensions', 'Width (mm)'),
    ('Dimensions', 'Depth (mm)'),
    ('Bottom Reinforcement', 'Left (BL)'),
    ('Bottom Reinforcement', 'Middle (B)'),
    ('Bottom Reinforcement', 'Right (BR)'),
    ('Top Reinforcement', 'Left (TL)'),
    ('Top Reinforcement', 'Middle (T)'),
    ('Top Reinforcement', 'Right (TR)'),
    ('Shear links', 'Left (H)'),
    ('Shear links', 'Middle (J)'),
    ('Shear links', 'Right (K)'),
    ('Side Face Reinforcement', '')
])
beam_schedule_df = pd.DataFrame(columns=columns)

## <center>For each index of the v4 respective df, populate the main beam_schedule_df with the necessary information</center>

In [716]:
beam_schedule_df['Storey'] = v4_flexural_df['TABLE:  Concrete Beam Flexure Envelope - ACI 318-19']
beam_schedule_df['Etabs ID'] = v4_flexural_df['ETABS beam ID']
beam_schedule_df['Dimensions', 'Width (mm)'] = v4_flexural_df['Width (mm)']
beam_schedule_df['Dimensions', 'Depth (mm)'] = v4_flexural_df['Depth (mm)']
beam_schedule_df.reset_index(drop=True, inplace=True)

# <center>Create v5 df for flexure, including rebar areas and torsional longitudinal rebar<center>

In [717]:
v5_flexural_df = v2_flexural_df.drop(['TABLE:  Concrete Beam Flexure Envelope - ACI 318-19', 'Unnamed: 1', 'Unnamed: 3'], axis=1)
v5_flexural_df = v5_flexural_df.rename(columns={'Unnamed: 4': 'Location', 'Unnamed: 7': 'Bottom reinforcement', 'Unnamed: 10': 'Top Reinforcement'})
v5_flexural_df.reset_index(drop=True, inplace=True)
v2_shear_df.reset_index(drop=True, inplace=True)
v5_flexural_df['Longitudinal tension rebar area (mm2)'] = v2_shear_df['Longitudinal tension rebar area (mm2)']

# <center> Create v6 df for flexure, including width and depth. This will be the df utilised for calculations.<center>

In [718]:
#create new columns
v5_flexural_df['Width (mm)'] = [1]*len(v5_flexural_df)
v5_flexural_df['Depth (mm)'] = [1]*len(v5_flexural_df)
v5_flexural_df['ETABS beam ID'] = [1]*len(v5_flexural_df)
#add new columns to beginning of dataframe, create v6
cols = ['ETABS beam ID', 'Width (mm)', 'Depth (mm)'] + [c for c in v5_flexural_df.columns if c not in ['ETABS beam ID', 'Width (mm)', 'Depth (mm)']]
v6_flexural_df = v5_flexural_df[cols]

In [719]:
v6_flexural_df.loc[:, 'Width (mm)'] = v2_flexural_df['Unnamed: 3'].apply(clean_width_dimensions)
v6_flexural_df.loc[:, 'Depth (mm)'] = v2_flexural_df['Unnamed: 3'].apply(clean_depth_dimensions)
v6_flexural_df['ETABS beam ID'] = v2_flexural_df['Unnamed: 1']
v6_flexural_df

,ETABS beam ID,Width (mm),Depth (mm),Location,Bottom reinforcement,Top Reinforcement,Longitudinal tension rebar area (mm2)
0,B213,300,500,End-I,3,28,0
1,B213,300,500,Middle,229,90,0
2,B213,300,500,End-J,302,124,0
3,B215,300,500,End-I,1683,2038,1878
4,B215,300,500,Middle,1356,1651,1878
5,B215,300,500,End-J,1357,1821,1878
6,B232,600,600,End-I,1563,1376,0
7,B232,600,600,Middle,690,998,0
8,B232,600,600,End-J,1388,1083,0
9,B235,300,500,End-I,14,85,0


In [720]:
# bottom_beam_rebar = {}
# values_per_key = 3
# values_counter = 0
# for i, row in v3_flexural_df.iterrows():
#     key = row['Unnamed: 1']
#     values = v2_flexural_df.iloc[values_counter:values_counter+values_per_key]['Unnamed: 7'].tolist()
#     bottom_beam_rebar[key] = values
#     values_counter += values_per_key

In [721]:
# top_beam_rebar = {}
# values_per_key = 3
# values_counter = 0
# for i, row in v3_flexural_df.iterrows():
#     key = row['Unnamed: 1']
#     values = v2_flexural_df.iloc[values_counter:values_counter+values_per_key]['Unnamed: 10'].tolist()
#     top_beam_rebar[key] = values
#     values_counter += values_per_key

In [722]:
# columns = pd.MultiIndex.from_tuples([
#     ('Etabs ID', ''),
#     ('Bottom Reinforcement', 'Left (BL)'),
#     ('Bottom Reinforcement', 'Middle (B)'),
#     ('Bottom Reinforcement', 'Right (BR)'),
#     ('Top Reinforcement', 'Left (TL)'),
#     ('Top Reinforcement', 'Middle (T)'),
#     ('Top Reinforcement', 'Right (TR)'),
# ])
# flexural_calc_df = pd.DataFrame(columns=columns)
# flexural_calc_df

In [723]:
#assign etabs beam id to flexural calc df
# flexural_calc_df['Etabs ID'] = v3_flexural_df['Unnamed: 1']

#turn dictionaries into dataframes
# top_beam_rebar = pd.DataFrame(top_beam_rebar)
# true_top_beam_rebar = top_beam_rebar.T
# bottom_beam_rebar = pd.DataFrame(bottom_beam_rebar)
# true_bottom_beam_rebar = bottom_beam_rebar.T
#combine datafrmes, with bottom rebar being first 3 columns
# v1_flexural_calc_df = pd.concat([bottom_beam_rebar, top_beam_rebar], axis=0)
# v2_flexural_calc_df = v1_flexural_calc_df.T
# v2_flexural_calc_df = v2_flexural_calc_df.rename(columns={'213': 'Bottom reinforcement (mm)'})
# v2_flexural_calc_df